Downloading the files

In [ ]:
# !wget "https://raw.githubusercontent.com/MarsPanther/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_E-Bible/amharic.txt" -O amharic_bible.txt
# !wget "https://raw.githubusercontent.com/MarsPanther/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_E-Bible/english.txt" -O english_bible.txt

# !wget "https://raw.githubusercontent.com/MarsPanther/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_History/amharic.txt" -O amharic_history.txt
# !wget "https://raw.githubusercontent.com/MarsPanther/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_History/english.txt" -O english_history.txt

# !wget "https://raw.githubusercontent.com/MarsPanther/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_Legal/amharic.txt" -O amharic_legal.txt
# !wget "https://raw.githubusercontent.com/MarsPanther/Amharic-English-Machine-Translation-Corpus/master/Amharic_English_Legal/english.txt" -O english_legal.txt


Loading and combining

In [ ]:
# def load_texts(amharic_files, english_files):
#     amharic_data = []
#     english_data = []
    
#     for am_file, en_file in zip(amharic_files, english_files):
#         with open(am_file, "r", encoding="utf-8") as am_f, open(en_file, "r", encoding="utf-8") as en_f:
#             amharic_data.extend(am_f.readlines())
#             english_data.extend(en_f.readlines())
    
#     return amharic_data, english_data

# # List of all Amharic and English files
# amharic_files = ["amharic_bible.txt", "amharic_history.txt", "amharic_legal.txt"]
# english_files = ["english_bible.txt", "english_history.txt", "english_legal.txt"]

# # Load and combine datasets
# amharic_sentences, english_sentences = load_texts(amharic_files, english_files)

# # Check if lengths match
# assert len(amharic_sentences) == len(english_sentences), "Mismatch in dataset sizes!"

# # Print sample of the combined data
# print("Amharic:", amharic_sentences[0].strip())
# print("English:", english_sentences[0].strip())


Tokenization

In [ ]:
# # from transformers import BertTokenizer
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # Load tokenizer (can be adjusted if you choose a different model)
# # tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
# # Tokenize Amharic and English sentences
# def tokenize_data(amharic_sentences, english_sentences):
#     amharic_encodings = tokenizer(amharic_sentences, padding=True, truncation=True, return_tensors="pt")
#     english_encodings = tokenizer(english_sentences, padding=True, truncation=True, return_tensors="pt")
#     return amharic_encodings, english_encodings

# # Tokenize the data
# amharic_encodings, english_encodings = tokenize_data(amharic_sentences, english_sentences)

# # Check tokenized output
# print(amharic_encodings['input_ids'][0])
# print(english_encodings['input_ids'][0])


Prepare the datasett in the format the model can work with

In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# # from transformers import MarianTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M")

# # Load the tokenizer for MarianMT (Amharic-English)
# # tokenizer = MarianTokenizer.from_pretrained("facebook/m2m100_418M")


# # Define a custom dataset class
# class TranslationDataset(Dataset):
#     def __init__(self, amharic_file, english_file, tokenizer, max_length=128):
#         # Read Amharic and English sentences from files
#         self.amharic_lines = open(amharic_file, 'r', encoding='utf-8').readlines()
#         self.english_lines = open(english_file, 'r', encoding='utf-8').readlines()
#         self.tokenizer = tokenizer
#         self.max_length = max_length
    
#     def __len__(self):
#         return len(self.amharic_lines)
    
#     def __getitem__(self, idx):
#         amharic_text = self.amharic_lines[idx].strip()
#         english_text = self.english_lines[idx].strip()

#         # Tokenize Amharic and English sentences
#         amharic_tokens = self.tokenizer(amharic_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
#         english_tokens = self.tokenizer(english_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")

#         # Extract input IDs and attention masks
#         input_ids = amharic_tokens['input_ids'].squeeze()  # Remove the batch dimension
#         attention_mask = amharic_tokens['attention_mask'].squeeze()
#         labels = english_tokens['input_ids'].squeeze()

#         return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

# # Load the dataset
# dataset = TranslationDataset('amharic_bible.txt', 'english_bible.txt', tokenizer)

# # Create a DataLoader
# batch_size = 8
# train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Example: check the tokenized output of one batch
# for batch in train_dataloader:
#     print(batch['input_ids'].shape)
#     print(batch['attention_mask'].shape)
#     print(batch['labels'].shape)
#     break


Model setup

In [ ]:
# from transformers import MarianMTModel
# from torch.optim import AdamW
# import torch

# # Load the pre-trained MarianMT model
# # model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-afa')
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M")

# # Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Define optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)


Training Function

In [ ]:
# from tqdm import tqdm

# # Training function
# def train(model, dataloader, optimizer, device, epochs=3):
#     model.train()  # Set model to training mode
    
#     for epoch in range(epochs):
#         total_loss = 0
#         progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        
#         for batch in progress_bar:
#             optimizer.zero_grad()  # Zero gradients from the previous step
            
#             # Move tensors to the appropriate device
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)

#             # Forward pass
#             outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss
#             total_loss += loss.item()

#             # Backward pass
#             loss.backward()

#             # Update weights
#             optimizer.step()

#             # Update the progress bar
#             progress_bar.set_postfix({"loss": loss.item()})

#         # Print average loss for the epoch
#         avg_loss = total_loss / len(dataloader)
#         print(f"Epoch {epoch+1} Average Loss: {avg_loss:.4f}")

# # Train the model
# train(model, train_dataloader, optimizer, device)


Saving the model

In [ ]:
# # Saving the trained model
# model.save_pretrained('trained_amaric_english_model')
# tokenizer.save_pretrained('trained_amaric_english_model')


Evaluation Function

In [ ]:
# # Evaluation function
# def evaluate(model, dataloader, device):
#     model.eval()  # Set model to evaluation mode
#     total_loss = 0

#     with torch.no_grad():  # Disable gradient calculation
#         for batch in dataloader:
#             # Move tensors to the appropriate device
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)

#             # Forward pass
#             outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss
#             total_loss += loss.item()

#     # Print average evaluation loss
#     avg_loss = total_loss / len(dataloader)
#     print(f"Evaluation Loss: {avg_loss:.4f}")

# # Assuming you have a validation DataLoader for evaluation
# # Evaluate the model
# evaluate(model, train_dataloader, device)


Translation Function

In [ ]:
# # Translation function for inference
# def translate(model, tokenizer, text, device):
#     model.eval()  # Set model to evaluation mode

#     # Tokenize the input text
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
#     input_ids = inputs['input_ids'].to(device)
#     attention_mask = inputs['attention_mask'].to(device)

#     # Generate translation
#     with torch.no_grad():
#         generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, num_beams=5, max_length=128)

#     # Decode the generated tokens
#     translated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
#     return translated_text

# # Example translation
# test_sentence = "እንዴት ነህ?"
# translated = translate(model, tokenizer, test_sentence, device)
# print("Original:", test_sentence)
# print("Translated:", translated)
